In [112]:
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier
import utils as ut
from sklearn.cross_validation import cross_val_score, KFold
from sklearn.metrics import roc_auc_score
from sklearn.lda import LDA
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale, MinMaxScaler

reload(ut)

X, y = ut.load_X(), ut.load_y()
# X = scale(X)

mms = MinMaxScaler()
X = mms.fit_transform(X)

#pca = PCA(n_components=7)
#pca.fit(X)
#print pca.explained_variance_ratio_

#X = pca.transform(X)

lda = LDA(store_covariance=True)
lda.fit(X, y)
print lda.covariance_

rfc = ExtraTreesClassifier()
gbc = GradientBoostingClassifier()

tots = np.zeros(shape=(10, 3))

for j in range(10):
    kfold = KFold(X.shape[0], n_folds=5, shuffle=True, random_state=0)

    gbcs = np.zeros(len(kfold))
    rfcs = np.zeros(len(kfold))
    scores = np.zeros(len(kfold))

    for i, (train, test) in enumerate(kfold):
        rfc.fit(X[train], y[train])
        rfc_y = rfc.predict_proba(X[test])[:, 1]
        gbc.fit(X[train], y[train])
        gbc_y = gbc.predict_proba(X[test])[:, 1]
        # print roc_auc_score(y[test], rfc_y)
        # print roc_auc_score(y[test], gbc_y)

        alpha = 0.25

        tot = alpha*(rfc_y)+(1-alpha)*gbc_y
        # print tot
        # tot[tot>.95] = 1.0

        # print "power combined:\n%s\n" % roc_auc_score(y[test], tot)
        gbcs[i] = roc_auc_score(y[test], gbc_y)
        rfcs[i] = roc_auc_score(y[test], rfc_y)
        scores[i] = roc_auc_score(y[test], tot)
    
    tots[j] = np.array([gbcs.mean(), rfcs.mean(), scores.mean()])
    print j+1

print tots.mean(axis=0)

#print cross_val_score(etc, X, y, scoring='roc_auc', cv=5).mean()
#print cross_val_score(dtc, X, y, scoring='roc_auc', cv=5).mean()
#print cross_val_score(rfc, X, y, scoring='roc_auc', cv=5).mean()
#print cross_val_score(gbc, X, y, scoring='roc_auc', cv=5).mean()


[[  5.01979841e-03   1.98218398e-03   2.99394933e-03   1.59539700e-03
    7.65417503e-04   7.90739559e-04  -8.99592892e-04  -1.12864194e-03
    7.48235499e-04]
 [  1.98218398e-03   1.79253680e-03   1.04532102e-03   1.07505576e-03
    5.27602922e-04   9.82527564e-04  -1.77343739e-04  -2.27515539e-04
    7.18108686e-04]
 [  2.99394933e-03   1.04532102e-03   2.95982223e-03   6.20186849e-04
    3.12877948e-04   4.27340447e-04  -6.96983983e-04  -8.54432246e-04
    1.87448937e-04]
 [  1.59539700e-03   1.07505576e-03   6.20186849e-04   1.06597348e-03
    3.92633115e-04   4.44305248e-04  -1.40452074e-04  -1.80477547e-04
    6.56987215e-04]
 [  7.65417503e-04   5.27602922e-04   3.12877948e-04   3.92633115e-04
    8.91682041e-04   4.16561091e-04  -1.63848214e-04  -2.01847710e-04
    2.76536488e-04]
 [  7.90739559e-04   9.82527564e-04   4.27340447e-04   4.44305248e-04
    4.16561091e-04   8.68695125e-04  -5.11365378e-05  -6.58793456e-05
    4.23442202e-04]
 [ -8.99592892e-04  -1.77343739e-04  -6.